# **If you are using Colab for free, we highly recommend you active the T4 GPU hardware accelerator. Our models are designed to run with at least 16GB of RAM, activating T4 will grant the notebook 16GB of GDDR6 RAM as opposed to the 13GB Colab gives automatically.**
# **To active T4:**
# **1.   click on the "Runtime" tab**
# **2.   click on "Change runtime type"**
# **3.   select T4 GPU under Hardware Accelerator**
# **NOTE: there is a weekly usage limit on using T4 for free**


In [1]:
!pip install llmware

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 669.1/669.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 16.4 MB/s eta 0:00:00
ERROR: Operation cancelled by user


In [2]:
import os
from llmware.setup import Setup
from llmware.library import Library
from llmware.prompts import Prompt, HumanInTheLoop
from llmware.retrieval import Query
from llmware.configs import LLMWareConfig

ModuleNotFoundError: No module named 'llmware'

In [ ]:
LLMWareConfig().set_active_db("sqlite")
llm = "llmware/dragon-yi-6b-gguf"

In [ ]:
local_path = Setup().load_sample_files()
agreements_path = os.path.join(local_path, "AgreementsLarge")

In [ ]:
print(f"\nStarting:  Parsing 'AgreementsLarge' Folder")
msa_lib = Library().create_new_library("example6_library")
msa_lib.add_files(agreements_path)

In [ ]:
print(f"\nCompleted Parsing - now, let's look for the 'master service agreements', e.g., 'msa'")
q = Query(msa_lib)
query = '"master services agreement"'
results = q.text_search_by_page(query, page_num=1, results_only=False)

In [ ]:
msa_docs = results["file_source"]
msa_doc_ids = results["doc_ID"]
prompter = Prompt().load_model(llm)
print("update: identified the following msa doc id: ", msa_doc_ids)

In [3]:
for i, doc_id in enumerate(msa_doc_ids):

    print("\n")
    docs = msa_docs[i]
    if os.sep in docs:
      docs = docs.split(os.sep)[-1]

    print (i+1, "Reviewing MSA - ", doc_id, docs)

    doc_filter = {"doc_ID": [doc_id]}
    termination_provisions = q.text_query_with_document_filter("termination", doc_filter)

    sources = prompter.add_source_query_results(termination_provisions)

    response = prompter.prompt_with_source("What is the notice for termination for convenience?")

    stats = prompter.evidence_comparison_stats(response)
    ev_source = prompter.evidence_check_sources(response)

    for i, resp in enumerate(response):
      print("update: llm response - ", resp)
      print("update: compare with evidence- ", stats[i]["comparison_stats"])
      print("update: sources - ", ev_source[i]["source_review"])

    prompter.clear_source_materials()

NameError: name 'msa_doc_ids' is not defined

In [ ]:
print("\nupdate: Prompt state saved at: ", os.path.join(LLMWareConfig.get_prompt_path(),prompter.prompt_id))
prompter.save_state()

In [ ]:
csv_output = HumanInTheLoop(prompter).export_current_interaction_to_csv()
print("\nupdate: CSV output for human review - ", csv_output)